# Packages

In [1]:
import numpy as np
import pandas as pd
import astropy.units as au
from astropy.coordinates import SkyCoord, FK5

# Ancillary functions

In [2]:
# compute relative coordinates from R.A., Dec., and radius
def get_relative_coords(ra, dec, rgal):
    
    """
    Takes right ascension (in deg), declination (in deg) and
    galactocentric radius (in arbitrary unit) and returns right ascension
    and declination relative to the galactic center.
    """
    
    ind_center = np.where(rgal == 0)[0][0]  # get index of center
    coords_ref = [str(ra[ind_center]) + ' ' + str(dec[ind_center])]
    skycoords_ref = SkyCoord(coords_ref, frame=FK5, unit=(au.deg, au.deg))
    coords_map = (ra, dec) *au.deg
    skycoords_map = SkyCoord(ra=coords_map[0], dec=coords_map[1], frame=FK5)
    aframe = skycoords_ref.skyoffset_frame()
    delta_ra = skycoords_map.transform_to(aframe).lon.arcsec
    delta_dec = skycoords_map.transform_to(aframe).lat.arcsec
    
    return delta_ra, delta_dec

# Setup

In [3]:
# directories
wd = '/Users/lneumann/Documents/'
data_dir = wd + 'Data/'

# ALMOND
almond_galaxies = ['ngc0628','ngc1097','ngc1365','ngc1385','ngc1511','ngc1546','ngc1566','ngc1672','ngc1792','ngc2566',
                   'ngc2903','ngc2997','ngc3059','ngc3521','ngc3621','ngc4303','ngc4321','ngc4535','ngc4536','ngc4569',
                   'ngc4826','ngc5248','ngc5643','ngc6300','ngc7496']
almond_res_list = ['18.6as', '19.4as', '20.6as', '19.9as', '17.6as', '18.9as', '19.7as', '18.2as', '18.7as', '18.5as',
                   '18.3as', '20.4as', '16.7as', '21.1as', '18.9as', '20.2as', '19.6as', '22.8as', '21.5as', '19.2as',
                   '18.7as', '19.9as', '18.0as', '17.7as', '17.9as']

# EMPIRE
empire_galaxies = ['ngc0628', 'ngc2903', 'ngc3184', 'ngc3627', 'ngc4254', 'ngc4321', 'ngc5055', 'ngc5194', 'ngc6946']
empire_res_list = ['33.3as', '33.3as', '33.3as', '33.3as', '33.3as', '33.3as', '33.3as', '33.3as', '33.3as']


# combine
galaxy_list = almond_galaxies + empire_galaxies
res_list = almond_res_list + empire_res_list
survey_list = ['ALMOND'] * len(almond_galaxies) + ['EMPIRE'] * len(empire_galaxies)

# quantities
quants = ['CO10', #'SHUFFCO10', 
          'CO21', #'SHUFFCO21', 
          'HCN10', #'SHUFFHCN10', 
          'HCOP10', #'SHUFFHCOP10',
          'SD_MOL_SL24', 'SD_MOL_MW', 'SD_STAR', 'P_DE_SL24', 'P_DE_MW']

# Remove centre and save pystruct

In [4]:
# loop over galaxies
for glxy, res, survey in zip(galaxy_list, res_list, survey_list):

    ###### LOAD PYSTRUCTURE DATA ######

    # load pystructure database
    if survey == 'ALMOND':
        pystruct_file = data_dir + 'ALMOND/pystruct/' + '%s_data_struct_%s_spb2.npy' % (glxy, res)
    elif survey == 'EMPIRE':
        pystruct_file = data_dir + 'EMPIRE/pystruct/' + '%s_data_struct_%s_spb2.npy' % (glxy, res)
    struct = np.load(pystruct_file, allow_pickle = True).item()
    
    # galactocentric radius
    rgal_kpc = struct['rgal_kpc']
    rgal_as = struct['rgal_as']

    # coordinates
    ra_deg = struct['ra_deg']
    dec_deg = struct['dec_deg']
    delta_ra, delta_dec = get_relative_coords(ra_deg, dec_deg, rgal_as)
    
    # centre mask
    mask = np.sqrt(delta_ra**2 + delta_dec**2) <= 1.01*float(res[:-2])/2
    # print(sum(mask))

    # put data within central beam to nan
    for q in quants:
        x = np.array(struct['INT_VAL_'+q])
        x[mask] = np.nan
        struct['INT_VAL_'+q] = x
            
    ######## save new pystructure file ########
    fpath_new = pystruct_file[:-4] + '_wo_centre.npy'
    np.save(fpath_new, struct)